# Before your start:
- Read the README.md file
- Comment as much as you can and use the resources (README.md file)
- Happy learning!

- **Consider a significance level of 5% for all tests.**

In [1]:
# import numpy and pandas
import numpy as np
import pandas as pd


# Challenge 1 - Independent Sample T-tests

In this challenge, we will be using the Pokemon dataset. Before applying statistical methods to this data, let's first examine the data.

To load the data, run the code below.

In [3]:
# Run this code:

pokemon = pd.read_csv('../pokemon.csv')
pokemon.head(3)

,#,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
0,1,Bulbasaur,Grass,Poison,318,45,49,49,65,65,45,1,False
1,2,Ivysaur,Grass,Poison,405,60,62,63,80,80,60,1,False
2,3,Venusaur,Grass,Poison,525,80,82,83,100,100,80,1,False


Let's start off by looking at the `head` function in the cell below.

In [ ]:
# Your code here:

#code above

The first thing we would like to do is compare the legendary Pokemon to the regular Pokemon. To do this, we should examine the data further. What is the count of legendary vs. non legendary Pokemons?

In [4]:
# Your code here:
legendary = pokemon[pokemon['Legendary'] == True]
non_legendary = pokemon[pokemon['Legendary'] == False]
print(legendary["Legendary"].count())
print(non_legendary["Legendary"].count())


65
735


Compute the mean and standard deviation of the total points for both legendary and non-legendary Pokemon.

In [5]:
# Your code here:

legendary_total = legendary['Total'].mean()
legendary_total = legendary['Total'].std()

non_legendary_total= non_legendary['Total'].mean()
non_legendary_total= non_legendary['Total'].std()

The computation of the mean might give us a clue regarding how the statistical test may turn out; However, it certainly does not prove whether there is a significant difference between the two groups.

In the cell below, use the `ttest_ind` function in `scipy.stats` to compare the the total points for legendary and non-legendary Pokemon. Since we do not have any information about the population, assume the variances are not equal.

In [6]:
# Your code here:

from scipy.stats import ttest_ind

leg_sample = legendary["Total"]
non_leg_sample = non_legendary["Total"]

ttest_ind(leg_sample, non_leg_sample, equal_var=False)

TtestResult(statistic=25.8335743895517, pvalue=9.357954335957444e-47, df=102.79988763435729)

What do you conclude from this test? Write your conclusions below.

In [ ]:
# Your conclusions here:
can reject the null hypothesis 


How about we try to compare the different types of pokemon? In the cell below, list the types of Pokemon from column `Type 1` and the count of each type.

In [7]:
# Your code here:
type1_pokemon = pokemon["Type 1"].value_counts()
type1_pokemon.head()


Type 1
Water      112
Normal      98
Grass       70
Bug         69
Psychic     57
Name: count, dtype: int64

Since water is the largest group of Pokemon, compare the mean and standard deviation of water Pokemon to all other Pokemon.

In [8]:
# Your code here:
type_water = type1_pokemon["Water"].mean()
type_water = type1_pokemon["Water"].std()


types = ["Normal", "Grass", "Bug", "Psychic", "Fire", "Electric", "Rock", "Dragon", "Ground", "Ghost", "Dark", "Poison", "Steel", "Fighting", "Ice", "Fairy", "Flying"]

type1_pokemon_selected = pokemon[pokemon["Type 1"].isin(types)]

type_others = type1_pokemon_selected["Total"].mean()
type_others = type1_pokemon_selected["Total"].std()


Perform a hypothesis test comparing the mean of total points for water Pokemon to all non-water Pokemon. Assume the variances are equal. 

In [9]:
# Your code here:
water_pokemon = pokemon[pokemon["Type 1"] == "Water"]
non_water_pokemon = pokemon[pokemon["Type 1"] != "Water"]

water_total_points = water_pokemon["Total"]
non_water_total_points = non_water_pokemon["Total"]

ttest_ind(water_total_points, non_water_total_points, equal_var=True)



TtestResult(statistic=-0.4418547448849676, pvalue=0.6587140317488793, df=798.0)

Write your conclusion below.

In [ ]:
# Your conclusions here:
#do not reject the hypothesis testing


# Challenge 2 - Matched Pairs Test

In this challenge we will compare dependent samples of data describing our Pokemon. Our goal is to see whether there is a significant difference between each Pokemon's defense and attack scores. Our hypothesis is that the defense and attack scores are equal. In the cell below, import the `ttest_rel` function from `scipy.stats` and compare the two columns to see if there is a statistically significant difference between them.

In [10]:
# Your code here:

from scipy.stats import ttest_rel
#H0: There is no significant difference between the Attack and Defense stats, Attack = Deffense.
#H1: There is a significant difference between the Attack and Defense stats, Attack != Deffense.
ttest_rel(pokemon["Attack"], pokemon["Defense"])

TtestResult(statistic=4.325566393330478, pvalue=1.7140303479358558e-05, df=799)

Describe the results of the test in the cell below.

In [ ]:
# Your conclusions here:
# reject the hypothesis testing


We are also curious about whether therer is a significant difference between the mean of special defense and the mean of special attack. Perform the hypothesis test in the cell below. 

In [11]:
# Your code here:

#H0: There is no significant difference between the Sp. Atk and Sp. Def stats, Sp. Atk = Sp.Def.
#H1: There is a significant difference between the Sp. Atk and Sp. Def stats, Sp. Atk != Sp. Def.
ttest_rel(pokemon["Sp. Atk"], pokemon["Sp. Def"])


TtestResult(statistic=0.853986188453353, pvalue=0.3933685997548122, df=799)

Describe the results of the test in the cell below.

In [ ]:
# Your conclusions here:

#We reject the null hypothesis. There is enough evidence to suggest that the population mean is significantly different from the hypothesized value.

As you may recall, a two sample matched pairs test can also be expressed as a one sample test of the difference between the two dependent columns.

Import the `ttest_1samp` function and perform a one sample t-test of the difference between defense and attack. Test the hypothesis that the difference between the means is zero. Confirm that the results of the test are the same.

In [12]:
# Your code here:
# Your code here:
from scipy.stats import ttest_1samp

difference_scores = pokemon["Defense"] - pokemon["Attack"]

# H0: The mean difference between Defense and Attack scores is zero.
# H1: The mean difference between Defense and Attack scores is not zero.
ttest_1samp(difference_scores, popmean=0)


TtestResult(statistic=-4.325566393330478, pvalue=1.7140303479358558e-05, df=799)

# Bonus Challenge - The Chi-Square Test

The Chi-Square test is used to determine whether there is a statistically significant difference in frequencies. In other words, we are testing whether there is a relationship between categorical variables or rather when the variables are independent. This test is an alternative to Fisher's exact test and is used in scenarios where sample sizes are larger. However, with a large enough sample size, both tests produce similar results. Read more about the Chi Squared test [here](https://en.wikipedia.org/wiki/Chi-squared_test).

In the cell below, create a contingency table using `pd.crosstab` comparing whether a Pokemon is legenadary or not and whether the Type 1 of a Pokemon is water or not.

In [ ]:
# Your code here:



Perform a chi-squared test using the `chi2_contingency` function in `scipy.stats`. You can read the documentation of the function [here](https://docs.scipy.org/doc/scipy-0.15.1/reference/generated/scipy.stats.chi2_contingency.html).

In [ ]:
# Your code here:



Based on a 95% confidence, should we reject the null hypothesis?

In [ ]:
# Your answer here:

